In [34]:
import pandas as pd
from sklearn import preprocessing, model_selection, neighbors
import joblib
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance as ds
import numpy as np
import statsmodels as sms
import re
import csv
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import scipy.stats as stats
from scipy import optimize
import sys
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import patsy 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

In [35]:
'''2024: THIS IS ORIGINALLY FOR CONTENT-FILTERING PORTION, see Paul_Music.ipynb from Social Networks to see how it all fits together '''

'2024: THIS IS ORIGINALLY FOR CONTENT-FILTERING PORTION, see Paul_Music.ipynb from Social Networks to see how it all fits together '

In [36]:
# 🍄 Find the current file path
import os 
current_path = os.getcwd() 
print(current_path)

/Users/jenniferwang/musipy_by_jennwang


In [37]:
## merged files

big5music = pd.read_csv('/Users/jenniferwang/musipy_by_jennwang/data/big5_music_fixed.csv')
big5music

,userid,ope,con,ext,agr,neu,country_of_residence,q1,q2,q3,...,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50
0,07fe9ee33fb4242d41506c3a7c4f2614,4.75,4.10,3.40,3.90,2.35,US,4,4,3,...,1,1,1,1,1,3,2,2,3,3
1,96a80e9c310bfa7218a424432bea8830,3.85,2.65,2.30,3.65,3.70,US,6,6,5,...,0,0,6,0,0,9,0,0,0,0
2,692b2a7bcbad9cf502f5d69ed04f10ce,3.65,3.25,3.65,3.25,2.35,-1,3,3,6,...,6,6,6,6,6,6,4,4,7,5
3,1c1f96726df5643500c429b4b46066b9,4.60,3.90,2.80,3.00,3.00,US,5,5,6,...,7,8,5,5,6,5,4,5,6,8
4,cfb9728b6201df4120e348f441487ee3,3.40,2.65,2.25,2.75,3.30,US,7,5,9,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21297,026f3ec0ef1a4cb7f295227b932031b6,4.10,2.35,1.75,3.35,3.74,BR,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21298,4eb203be92d23dd7b08910d7883d3745,4.65,2.90,3.25,2.55,3.35,-1,8,8,9,...,0,0,0,0,0,0,0,0,0,0
21299,b66b3c8bbc513c44ed42510a2d2c89de,4.53,3.95,3.90,2.70,3.16,US,1,1,8,...,0,0,0,0,0,0,0,0,0,0
21300,ab071d6fdfc0e14256fdcb69a2dd777e,4.05,3.25,3.40,3.50,2.95,-1,8,8,7,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# correlation 
big5music.corr()

,ope,con,ext,agr,neu,q1,q2,q3,q4,q5,...,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50
ope,1.000000,0.028891,0.178437,0.055605,-0.067748,0.104047,0.059639,0.083532,0.039824,0.061170,...,0.097285,0.098596,0.077869,0.104916,0.096700,0.127958,0.045724,0.064121,0.097574,0.122503
con,0.028891,1.000000,0.221703,0.187076,-0.345658,0.045903,0.056829,0.037733,0.068342,0.024977,...,-0.011540,-0.009936,-0.038126,-0.022200,-0.021515,-0.005347,0.032046,0.016518,0.007121,0.005808
ext,0.178437,0.221703,1.000000,0.222488,-0.395474,-0.021643,-0.014502,-0.000310,-0.004912,0.029465,...,-0.043527,-0.030818,-0.045480,-0.026797,-0.032016,-0.010030,-0.013708,-0.019391,-0.023785,-0.018415
agr,0.055605,0.187076,0.222488,1.000000,-0.372284,0.076770,0.080474,0.048733,0.087349,0.087107,...,0.014452,0.013665,-0.031636,0.025769,0.015437,0.042362,0.058174,0.052412,0.039523,0.064770
neu,-0.067748,-0.345658,-0.395474,-0.372284,1.000000,-0.012604,-0.038431,-0.031597,-0.057527,-0.014558,...,0.011216,-0.000389,0.030089,0.010502,0.008892,-0.024272,-0.019980,-0.020113,-0.023741,-0.002138
q1,0.104047,0.045903,-0.021643,0.076770,-0.012604,1.000000,0.728705,0.622414,0.664484,0.581315,...,0.292287,0.263333,0.179308,0.284321,0.264068,0.281628,0.377007,0.342978,0.299797,0.412458
q2,0.059639,0.056829,-0.014502,0.080474,-0.038431,0.728705,1.000000,0.626074,0.722577,0.570545,...,0.252183,0.223656,0.136334,0.239435,0.217599,0.255944,0.376799,0.345408,0.280635,0.372141
q3,0.083532,0.037733,-0.000310,0.048733,-0.031597,0.622414,0.626074,1.000000,0.620511,0.613685,...,0.319347,0.277474,0.214824,0.308541,0.278642,0.270225,0.354573,0.311990,0.271157,0.372096
q4,0.039824,0.068342,-0.004912,0.087349,-0.057527,0.664484,0.722577,0.620511,1.000000,0.559689,...,0.245114,0.212003,0.130469,0.224508,0.205988,0.245490,0.372835,0.339442,0.280291,0.344826
q5,0.061170,0.024977,0.029465,0.087107,-0.014558,0.581315,0.570545,0.613685,0.559689,1.000000,...,0.283267,0.272186,0.207378,0.293264,0.272138,0.307027,0.373178,0.345240,0.276645,0.391808


In [39]:
# personality is from 1 to 5 
# song ratings is from 1 to 7 
mean = big5music.mean()
mean

ope    4.033824
con    3.351190
ext    3.218952
agr    3.524666
neu    2.831006
q1     4.635433
q2     3.940334
q3     4.455779
q4     3.583044
q5     4.071777
q6     3.390902
q7     3.398601
q8     3.041498
q9     3.937001
q10    3.138579
q11    3.518965
q12    4.578678
q13    4.229885
q14    3.901934
q15    4.052624
q16    5.155244
q17    4.985353
q18    4.200075
q19    4.275185
q20    4.615388
q21    2.428035
q22    2.842081
q23    3.335649
q24    3.417519
q25    2.832504
q26    2.353300
q27    2.109286
q28    2.008309
q29    2.578396
q30    2.043846
q31    1.974932
q32    2.058727
q33    1.959628
q34    1.725284
q35    1.877007
q36    2.711623
q37    2.115013
q38    1.962680
q39    2.357807
q40    1.718289
q41    2.833349
q42    2.782415
q43    2.135199
q44    2.608863
q45    2.730917
q46    1.881889
q47    2.150925
q48    1.699089
q49    2.091494
q50    2.445029
dtype: float64

# Content Filtering- kNN 

# PERSONALITY: Get DISTANCE (from SciPy) via COSINE DISTANCE to compare personality traits --> It is 1 - Cosine Similarity

In [40]:
# get distances from scipy distance function (ds)
# v is the input value (what the user is putting in)
def get_distances(row, new_row): 
    u = [row['ope'], row['agr'], row['neu'], row['con'], row['ext']]
    v = [new_row['ope'], new_row['agr'], new_row['neu'], new_row['con'], new_row['ext']] 
    return ds.cosine(u,v)

In [41]:
"""
X = Personality Dimensions 
Y = Ratings of the songs 
"""

feature_columns = ['ope', 'con', 'ext', 'agr', 'neu']
X = big5music[feature_columns]

target_columns = [f'q{i}' for i in range(1, 51)]
y = big5music[target_columns]

## Using KNeighborsRegressor (as kNN)!

In [42]:
knn = KNeighborsRegressor(n_neighbors= 2000, weights='distance', n_jobs=-1)
knn.fit(X,y)

KNeighborsRegressor(n_jobs=-1, n_neighbors=2000, weights='distance')

## Pickle the data

In [43]:
joblib.dump(knn, 'music1.pkl')

['music1.pkl']

## Checking Score

In [44]:
knn.score(X,y)

0.9580834420324433

In [45]:
import sklearn
print(sklearn.__version__)


0.24.1


## Merging

In [47]:
import pandas as pd

# Load the datasets
big5music = pd.read_csv('/Users/jenniferwang/musipy_by_jennwang/data/big5_music_fixed.csv')
song_names = pd.read_csv('/Users/jenniferwang/musipy_by_jennwang/data/songs_names.csv')

# Create a mapping from 'variable' to 'song_name'
song_name_mapping = dict(zip(song_names['Variable'], song_names['Title']))

# Rename columns in big5music using the mapping
big5music_renamed = big5music.rename(columns=song_name_mapping)

# Print to verify
print(big5music_renamed.head())

                             userid   ope   con   ext   agr   neu  \
0  07fe9ee33fb4242d41506c3a7c4f2614  4.75  4.10  3.40  3.90  2.35   
1  96a80e9c310bfa7218a424432bea8830  3.85  2.65  2.30  3.65  3.70   
2  692b2a7bcbad9cf502f5d69ed04f10ce  3.65  3.25  3.65  3.25  2.35   
3  1c1f96726df5643500c429b4b46066b9  4.60  3.90  2.80  3.00  3.00   
4  cfb9728b6201df4120e348f441487ee3  3.40  2.65  2.25  2.75  3.30   

  country_of_residence  Children of Spring  Through the Years  Safety  ...  \
0                   US                   4                  4       3  ...   
1                   US                   6                  6       5  ...   
2                   -1                   3                  3       6  ...   
3                   US                   5                  5       6  ...   
4                   US                   7                  5       9  ...   

   Falling Down 2  Dick Dater  White Knuckles  Frequency of a Heartbeat  \
0               1           1            

In [52]:
output_path = '/Users/jenniferwang/musipy_by_jennwang/data/final.csv'
big5music_renamed.to_csv(output_path, index=False)